### Maximal Marginal Relevance
MMR (Maximal Marginal Relevance) is a powerful diversity-aware retrieval technique used in information retrieval and RAG pipelines to balance relevance and novelty when selecting documents.

In [61]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
# Document Loader (moved to langchain_community)
from langchain_community.document_loaders import TextLoader

# Text Splitter (moved to its own package)
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Chat model initializer (still valid, but provider-based is preferred)
from langchain.chat_models import init_chat_model

# Prompt templates (moved to langchain_core)
from langchain_core.prompts import PromptTemplate

# Chains (still valid locations)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough




In [62]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [63]:
# Step 1: Load and chunk the document
loader = TextLoader("langchain_rag_dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)
chunks

[Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain is an open-source framework designed to simplify the development of applications using large language models (LLMs).\nLangChain provides abstractions for working with prompts, chains, memory, and agents, making it easier to build complex LLM-based systems.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='The framework supports integration with various vector databases like FAISS and Chroma for semantic retrieval.\nLangChain enables Retrieval-Augmented Generation (RAG) by allowing developers to fetch relevant context before generating responses.'),
 Document(metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
 Document(metadata={'

In [64]:
# Step 2: FAISS Vector Store with HuggingFace Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding_model)


In [65]:
### Step 3: Create MMR Retirever
retriever=vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3}
)

In [71]:
import os
key = os.getenv("GROQ_API_KEY")
print(key is not None, key[:6] if key else None)


True gsk_Bt


In [66]:
# Step 4: Prompt and LLM
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
Answer the question based on the context provided.

Context:
{context}

Question: {input}
""")

llm=init_chat_model("groq:gemma2-9b-it")


In [67]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {
        "context": retriever,       
        "input": RunnablePassthrough()  
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [69]:
from dotenv import load_dotenv
load_dotenv()


True

In [ ]:
query = "How does LangChain support agents and memory?"
response = rag_chain.invoke(query)

print("✅ Answer:\n", response)


In [ ]:
response

{'input': 'How does LangChain support agents and memory?',
 'context': [Document(id='27a1531f-1d37-4619-ad03-70b21cdf56b2', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='Memory in LangChain helps models retain previous interactions, making multi-turn conversations more coherent.\nAgents in LangChain can use tools like calculators, search APIs, or custom functions based on the instructions they receive.'),
  Document(id='fcd7afd5-b72a-4822-8ebe-c75d4e9250e0', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain agents can interact with external APIs and databases, enhancing the capabilities of LLM-powered applications.\nRAG pipelines in LangChain involve document loading, splitting, embedding, retrieval, and LLM-based response generation.'),
  Document(id='e755e7c1-f863-4f8c-b97b-7b6d7281e29f', metadata={'source': 'langchain_rag_dataset.txt'}, page_content='LangChain allows LLMs to act as agents that decide which tool to call and in what order duri